# Debugging the CW308_STM32F3 Using ChipWhisperer

## Background

When testing software, it can be extremely helpful to view and modify the state of the software while it is running. For programs running on fully featured operating systems (Windows, Mac, Linux, etc.), this can be accomplished by using debugging programs, such as `gdb`. Microcontrollers and FPGAs, on the other hand, require additional hardware to debug. While it is better to use a device dedicated to debugging, beginning with ChipWhisperer 5.6.1, you can use your ChipWhisperer capture board to debug target devices. This is done over SWD/JTAG, which most devices support. To see if your target supports JTAG/SWD, you should check the reference manual/datasheet for its microcontroller.

This demo will run will show you how to set your hardware up for debugging, how to set the software up for debugging, and finally how to program and debug an STM32F3. In addition to this demo, you may want to skip through and have open our [debugging documentation](https://chipwhisperer.readthedocs.io/en/latest/debugging.html).

## Checking for ChipWhisperer Support

This demo requires ChipWhisperer 5.6.1. In addition, you must have firmware from this version of ChipWhisperer or newer running on your device. Run the following line to check for support:

In [1]:
import sys
if '../../software' not in sys.path:
    sys.path.append('../../software')
if '../courses/fault101/python-dilithium/' not in sys.path:
    sys.path.append('../courses/fault101/python-dilithium/')
sys.path

['/home/alexis/chipwhisperer.dilithium/jupyter/demos',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/home/alexis/.venvs/chipwhisperer.dilithium/lib/python3.8/site-packages',
 '../../software',
 '../courses/fault101/python-dilithium/']

In [2]:
import os
print(os.environ['PATH'])
if '/home/alexis/bin' not in os.environ['PATH'].split(':'):
    os.environ['PATH'] += ':/home/alexis/bin'
print(os.environ['PATH'])

/home/alexis/.venvs/chipwhisperer.dilithium/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin
/home/alexis/.venvs/chipwhisperer.dilithium/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/alexis/bin


In [3]:
import chipwhisperer as cw
if cw.__version__ < '5.6.1':
    print("MPSSE not supported.")
else:
    scope = cw.scope()
    if scope.check_feature("MPSSE"):
        print("MPSSE supported!")
    else:
        print("MPSSE not supported.")
    scope.dis()

MPSSE supported!


## Software Prerequisites

In addition to ChipWhisperer, you will need the following software:

1. OpenOCD, available at https://openocd.org/pages/getting-openocd.html. It's recommended that you download an OpenOCD binary or built it yourself, rather than getting it through a package manager, as the package manager will usually install an old version of OpenOCD
1. GDB for your device of choice (`arm-none-eabi-gdb`, in our case). This is typically included with the compiler you use to build firmware for your target.

Ensure both openocd and gdb are available on your path:

* Windows: https://www.computerhope.com/issues/ch000549.htm (set in user variables)
* Mac/Linux: https://superuser.com/questions/488173/how-can-i-edit-the-path-on-linux

Once you've got those installed, quit jupyter and reopen it in a new terminal window so that your path updates. Make sure you're able to access both of these programs:

In [4]:
%%bash
echo $PATH

/home/alexis/.venvs/chipwhisperer.dilithium/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/alexis/bin


In [5]:
%%bash
arm-none-eabi-gdb --version

Python Exception <class 'ModuleNotFoundError'>: No module named 'gdb'
arm-none-eabi-gdb: warning: 
Could not load the Python gdb module from `//share/gdb/python'.
Limited Python support is available from the _gdb module.
Suggest passing --data-directory=/path/to/gdb/data-directory.


GNU gdb (Arm GNU Toolchain 12.2.Rel1 (Build arm-12.24)) 12.1.90.20221210-git
Copyright (C) 2022 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.


In [6]:
%%bash
openocd --version

xPack OpenOCD x86_64 Open On-Chip Debugger 0.11.0+dev (2022-09-01-17:56)
Licensed under GNU GPL v2
For bug reports, read
	http://openocd.org/doc/doxygen/bugs.html


## Hardware Setup

### ChipWhisperer-Lite, Pro, Nano, or Husky with CW308T_STM32F3

1. Connect your ChipWhisperer capture board to your PC
1. Connect your capture board and CW308 via the CW 20-pin connector
1. Use jumper wires to connect the following pins on the CW308:
    1. J5 Pin 12 (SCK)  -> J8 Pin 6 (J_TCK)
    1. J5 Pin 18 (PDID) -> J8 Pin 5 (J_TMS)
    1. J5 Pin 13 (MISO) -> J8 Pin 4 (J_TDO)
    1. J5 Pin 14 (MOSI) -> J8 Pin 3 (J_TDI)
    
<img src="img/MPSSE_con_1x.jpg" alt="MPSSE-DIO" width=600>
    
### ChipWhisperer-Husky with CW308T_STM32F3 using 20-pin DIO Header

1. Connect your ChipWhisperer capture board to your PC
1. Connect your capture board and CW308 via the CW 20-pin connector
1. Connect the 20-pin DIO header to J6 on the CW308. Ensure the keyed part of the connector is facing away from the board

<img src="img/MPSSE_con_3.jpg" alt="MPSSE-DIO" width=600>

### ChipWhisperer-Lite, Pro, Nano, or Husky with CW312T_STM32F3

Coming soon!

## Software Setup

### Lite, Pro, Nano, or Husky (non DIO) Setup

All you need to do to enable MPSSE is connect to your scope and run `scope.enable_MPSSE(1)` (we'll also record the CW's USB PID for later usage):

In [7]:
import chipwhisperer as cw
scope = cw.scope()
PID = hex(scope._getNAEUSB().usbtx.device.getProductID())
scope.enable_MPSSE(1)
scope.dis()
print(PID)

(ChipWhisperer Scope ERROR|File naeusbchip.py:108) Scope already disconnected!


0xace2


###  Husky DIO Setup

Alternatively, if you want to use the ChipWhisperer-Husky's 20-pin DIO header, enable that before running `scope.enable_MPSSE(1)`:

In [8]:
# we do not have husky
#import chipwhisperer as cw
#scope = cw.scope()
#PID = hex(scope._getNAEUSB().usbtx.device.getProductID())
#scope.enable_MPSSE(1, husky_userio='jtag') # or husky_userio='swd'

With that done, we can now debug our target. Note that `scope.enable_MPSSE(1)` will disconnect your target in Python, meaning you'll have to reconnect to interact via the API. On Windows, only one process at a time can access a USB device, meaning if you reconnect in Python, you won't be able to connect in OpenOCD and vice versa.

We'll start by building and uploading some firmware to the target:

In [9]:
%%bash -s "$PID"

# build firmware
cd ../../hardware/victims/firmware/simpleserial-dilithium-ref
make PLATFORM=CW308_STM32F4 CRYPTO_TARGET=NONE CRYPTO_OPTIONS=NONE SS_VER=SS_VER_2_1 -j

# upload firmware via JTAG
openocd -s ~/opt/xpack-openocd-0.11.0-5/scripts -f ../../../../openocd/cw_openocd.cfg -c "transport select jtag" -c "ftdi vid_pid 0x2b3e $1" -f "target/stm32f4x.cfg" -c "init" -c "targets" -c "halt" -c "flash write_image erase simpleserial-aes-CW308_STM32F3.elf" -c "flash verify_image simpleserial-aes-CW308_STM32F3.elf" -c "reset run" -c "shutdown"

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
.
arm-none-eabi-gcc (15:9-2019-q4-0ubuntu1) 9.2.1 20191025 (release) [ARM/arm-9-branch revision 277599]
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Welcome to another exciting ChipWhisperer target build!!
.
Compiling:
-en     simpleserial-dilithium-ref.c ...
.
Compiling:
mkdir -p "objdir-CW308_STM32F4"
-en     .././simpleserial/simpleserial.c ...
.
mkdir -p "objdir-CW308_STM32F4"
Compiling:
-en     .././hal/stm32f4/stm32f4_hal.c ...
.
mkdir -p "objdir-CW308_STM32F4"
Compiling:
.
-en     .././hal/stm32f4/stm32f4_hal_lowlevel.c ...
Compiling:
.
-en     .././hal/stm32f4/stm32f4_sysmem.c ...
Compiling:
mkdir -p "objdir-CW308_STM32F4"
-en     .././hal/stm32f4/stm32f4xx_hal_rng.c ...
.
mkdir -p "objdir-CW308_STM32F4"
mkdir -p "objdi

In file included from .././hal/stm32f4/stm32f4_hal.c:3:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
-e Done!


In file included from .././hal/stm32f4/stm32f4_hal_lowlevel.c:39:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
LINKING:
-en     simpleserial-dilithium-ref-CW308_STM32F4.elf ...
-e Done!
.
.
.
Creating load file for Flash: simpleserial-dilithium-ref-CW308_STM32F4.hex
.
.
Creating load file for Flash: simpleserial-dilithium-ref-CW308_STM32F4.bin
Creating load file for EEPROM: simpleserial-dilithium-ref-CW308_STM32F4.eep
Creating Extended Listing: simpleserial-dilithium-ref-CW308_STM32F4.lss
Creating Symbol Table: simpleserial-dilithium-ref-CW308_STM32F4.sym
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-dilithium-ref-CW308_STM32F4.elf simpleserial-dilithium-ref-CW308_STM32F4.hex
arm-none-eabi-objcopy -O binary -R .eeprom -R .fuse -R .lock -R .signature simpleserial-dilithium-ref-CW308_STM32F4.elf simpleserial-dilithium-ref-CW308_STM32F4.bin
arm-none-eabi-objcopy -j .eeprom --set-section-flags=.eeprom="alloc,load" \
--change-section-lma .eeprom=0 --no-change-warnings -O ihex simpleserial-dilithium-ref-CW308_STM32F4.elf simpleserial-dilithium-ref-CW

xPack OpenOCD x86_64 Open On-Chip Debugger 0.11.0+dev (2022-09-01-17:56)
Licensed under GNU GPL v2
For bug reports, read
	http://openocd.org/doc/doxygen/bugs.html
none separate

jtag
Info : clock speed 2000 kHz
Error: JTAG scan chain interrogation failed: all zeroes
Error: Check JTAG interface, timings, target power, etc.
Error: Trying to use configured scan chain anyway...
Error: stm32f4x.cpu: IR capture error; saw 0x00 not 0x01
Warn : Bypassing JTAG setup events due to errors
Error: Invalid ACK (0) in DAP response




CalledProcessError: Command 'b'\n# build firmware\ncd ../../hardware/victims/firmware/simpleserial-dilithium-ref\nmake PLATFORM=CW308_STM32F4 CRYPTO_TARGET=NONE CRYPTO_OPTIONS=NONE SS_VER=SS_VER_2_1 -j\n\n# upload firmware via JTAG\nopenocd -s ~/opt/xpack-openocd-0.11.0-5/scripts -f ../../../../openocd/cw_openocd.cfg -c "transport select jtag" -c "ftdi vid_pid 0x2b3e $1" -f "target/stm32f4x.cfg" -c "init" -c "targets" -c "halt" -c "flash write_image erase simpleserial-aes-CW308_STM32F3.elf" -c "flash verify_image simpleserial-aes-CW308_STM32F3.elf" -c "reset run" -c "shutdown"\n'' returned non-zero exit status 1.

If you don't see any errors, then congratulations: You've now used JTAG to upload code onto your target! Next, let's do a little debugging instead. We'll begin by running openocd again, with some different commands:

In [ ]:
%%bash -s "$PID"

# build firmware
cd ../../hardware/victims/firmware/simpleserial-aes

openocd -f ../../../../openocd/cw_openocd.cfg -c "transport select jtag" -c "ftdi vid_pid 0x2b3e $1" -f "target/stm32f3x.cfg"

**NOTE: The above command will continuously run until you restart this jupyter notebook**

If we want to do some actual debugging, we'll need an interactive terminal. Any terminal with access to the correct gdb and openocd will work, but we'll use Jupyter's terminal here, as it already has everything setup:

![asdf](img/jupyter_terminal.png)

To start debugging, run the following commands in that terminal (you may need to adjust the paths if you don't start in the jupyter folder):

```bash
# try cd hardware/victims/firmware/simpleserial-aes if the command below doesn't work
cd ../hardware/victims/firmware/simpleserial-aes 
arm-none-eabi-gdb simpleserial-aes-CW308_STM32F3.elf -ex "target extended-remote localhost:3333" -ex "monitor reset halt" -ex "load"
```

You should now have an active gdb session up and running:

![](img/gdb_startup.png)

We won't do a full gdb tutorial, but here's some basic commands to get you up and running:
quit
* `help` - prints a useful help screen
* `bt` - prints all stack frames (i.e. all the functions from `main()` down to your current position)
* `s` - step program until it reaches the next source line (moves into function)
* `n` - step program, proceeding through subroutine calls (moves above functions)
* `l` - list source lines around current location

**Warning: the start and run commands may not properly restart the firmware. You can restart from the beginning of your code by running `load` followed by `s`**

![](img/using_gdb.png)

In [11]:
scope.dis()
target.dis()

(ChipWhisperer Scope ERROR|File naeusbchip.py:108) Scope already disconnected!


NameError: name 'target' is not defined